In [1]:
import numpy as np 
import time 
import math
from sklearn.model_selection import train_test_split

In [2]:
def loadData(filename):
    dataArr = [];labelArr = []
    fr = open(filename)
    #返回列表，包含所有的行。
    for line in fr.readlines():
        
        #获取当前行，并按“，”切割成字段放入列表中
        #strip：去掉每行字符串首尾指定的字符（默认空格或换行符）
        #split：按照指定的字符将字符串切割成每个字段，返回列表形式
        curLine = line.strip().split(',')
        #将每行中除标记外的数据放入数据集中（curLine[0]为标记信息）
        #在放入的同时将原先字符串形式的数据转换为整型
        #此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
        dataArr.append([int(int(num)>128) for num in curLine[1:]])
        #将标记信息放入标记集中
        #放入的同时将标记转换为整型
        labelArr.append(int(curLine[0]))
    #返回数据集和标记
    return dataArr,labelArr

In [3]:
def NaiveBayes(Py,Px_y,x):
    '''
    通过朴素贝叶斯进行概率估计
    :param Py: 先验概率分布
    :param Px_y: 条件概率分布
    :param x: 要估计的样本x
    :return: 返回所有label的估计概率
    '''
    #设置特征数目
    #len(X[0])==784
    
    featureNum = 784
    #设置类别数目
    classNum = 10
    
    #建立存放所有标记的估计概率数组
    #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    P = [0] * classNum
    
    #对于每一个类别，单独估计其概率
    for i in range(classNum):
        #初始化sum为0，sum为求和项。
        #在训练过程中对概率进行了log处理，所以这里原先应当是连乘所有概率，最后比较哪个概率最大
        #但是当使用log处理时，连乘变成了累加，所以使用sum
        sum = 0
        
        #获取每一个条件概率值，进行累加
        for j in range(featureNum):
            sum +=Px_y[i][j][x[j]]
        #最后再和先验概率相加（也就是式4.7中的先验概率乘以后头那些东西，乘法因为log全变成了加法）
        P[i] = sum+ Py[i]
    return P.index(max(P))
    

In [4]:
def model_test(Py,Px_y,testDataArr,testLabelArr):
    '''
    对测试集进行测试
    :param Py: 先验概率分布
    :param Px_y: 条件概率分布
    :param testDataArr: 测试集数据
    :param testLabelArr: 测试集标记
    :return: 准确率
    '''
    #错误值计数
    errorCnt = 0
    #循环遍历测试集中的每一个样本
    for i in range(len(testDataArr)):
        #获取预测值
        presict = NaiveBayes(Py,Px_y,testDataArr[i])
        #与答案进行比较
        if presict != testLabelArr[i]:
            #若错误  错误值计数加1
            errorCnt += 1
     #返回准确率   
    return 1-(errorCnt/len(testDataArr))

In [5]:
def getAllProbability(trainDataArr, trainLabelArr):
    '''
    通过训练集计算先验概率分布和条件概率分布
    :param trainDataArr: 训练数据集
    :param trainLabelArr: 训练标记集
    :return: 先验概率分布和条件概率分布
    '''
    #设置样本特诊数目，数据集中手写图片为28*28，转换为向量是784维。
    # （我们的数据集已经从图像转换成784维的形式了，CSV格式内就是）
    featureNum = 784
    #设置类别数目，0-9共十个类别
    classNum = 10
    #初始化先验概率分布存放数组，后续计算得到的P(Y = 0)放在Py[0]中，以此类推
    #数据长度为10行1列
    Py = np.zeros((classNum, 1))
    
    #对每个类别进行一次循环，分别计算它们的先验概率分布
    #计算公式为书中"4.2节 朴素贝叶斯法的参数估计 公式4.8"
    for i in range(classNum):
        #np.mat(trainLabelArr) == i：将标签转换为矩阵形式，里面的每一位与i比较，若相等，该位变为Ture，反之False
        #np.sum(np.mat(trainLabelArr) == i):计算上一步得到的矩阵中Ture的个数，进行求和(直观上就是找所有label中有多少个
        #为i的标记，求得4.8式P（Y = Ck）中的分子)
        #np.sum(np.mat(trainLabelArr) == i)) + 1：参考“4.2.3节 贝叶斯估计”，例如若数据集总不存在y=1的标记，也就是说
        #手写数据集中没有1这张图，那么如果不加1，由于没有y=1，所以分子就会变成0，那么在最后求后验概率时这一项就变成了0，再
        #和条件概率乘，结果同样为0，不允许存在这种情况，所以分子加1，分母加上K（K为标签可取的值数量，这里有10个数，取值为10）
        #参考公式4.11
        #(len(trainLabelArr) + 10)：标签集的总长度+10.
        #((np.sum(np.mat(trainLabelArr) == i)) + 1) / (len(trainLabelArr) + 10)：最后求得的先验概率
        Py[i] = ((np.sum(np.mat(trainLabelArr) == i)) + 1) / (len(trainLabelArr) + 10)
    #转换为log对数形式
    #log书中没有写到，但是实际中需要考虑到，原因是这样：
    #最后求后验概率估计的时候，形式是各项的相乘（“4.1 朴素贝叶斯法的学习” 式4.7），这里存在两个问题：1.某一项为0时，结果为0.
    #这个问题通过分子和分母加上一个相应的数可以排除，前面已经做好了处理。2.如果特征特别多（例如在这里，需要连乘的项目有784个特征
    #加一个先验概率分布一共795项相乘，所有数都是0-1之间，结果一定是一个很小的接近0的数。）理论上可以通过结果的大小值判断， 但在
    #程序运行中很可能会向下溢出无法比较，因为值太小了。所以人为把值进行log处理。log在定义域内是一个递增函数，也就是说log（x）中，
    #x越大，log也就越大，单调性和原数据保持一致。所以加上log对结果没有影响。此外连乘项通过log以后，可以变成各项累加，简化了计算。
    #在似然函数中通常会使用log的方式进行处理（至于此书中为什么没涉及，我也不知道）
    Py = np.log(Py) 
    
    #计算条件概率 Px_y= P(X=x|Y = y)
    #计算条件概率分成了两个步骤，下方第一个大for循环用于累加，参考书中“4.2.3 贝叶斯估计 式4.10”，下方第一个大for循环内部是
    #用于计算式4.10的分子，至于分子的+1以及分母的计算在下方第二个大For内
    
    
    #初始化为全0矩阵，用于存放所有情况下的条件概率
    Px_y = np.zeros((classNum, featureNum, 2))#(堆，行，列)(类别，特征，取值0或1)参数2列存放 二值化后的值 x[j] ，只有0 or 1
    #对 标记集 进行遍历
    for i in range(len(trainLabelArr)):
        
        #获取当前循环所使用的标记
        label = trainLabelArr[i]
        #获取当前要处理的样本
        x = trainDataArr[i]
        
        #对该样本的每一维特征进行遍历
        for j in range(featureNum):
            #在矩阵中对应位置加1
            #这里还没有计算条件概率，先把所有数累加，全加完以后，即统计个数，在后续步骤中再求对应的条件概率
            # 重点理解 x[j]，样本x 的第j个特征的取值
            Px_y[label][j][x[j]] += 1 #因为经过二值化，x[j]的取值只有 0 或 1
            
            
    #第二个大for，计算式4.10的分母，以及分子和分母之间的除法
    #循环每一个标记（共10个）
    for label in range(classNum):
        #循环每一个标记对应的每一个特征
        for j in range(featureNum):
            #获取y=label，第j个特征为0的个数
            Px_y0 = Px_y[label][j][0]
            #获取y=label，第j个特征为1的个数
            Px_y1 = Px_y[label][j][1]
            #对式4.10的分子和分母进行相除，再除之前依据贝叶斯估计，分母需要加上2（为每个特征可取值个数）
            #*******分别计算对于     y= label &&  x第j个特征为0和1的条件概率分布
            Px_y[label][j][0] = np.log((Px_y0 + 1) / (Px_y0 + Px_y1 + 2))
            Px_y[label][j][1] = np.log((Px_y1 + 1) / (Px_y0 + Px_y1 + 2))
        #返回先验概率分布和条件概率分布
    return Py, Px_y


In [6]:
Mat1 = np.ones((1,10))
Mat1.shape

(1, 10)

In [7]:
mat1 = np.log(Mat1)
mat1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
np.zeros((2,3,5))

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [9]:
start = time.time()
# 获取训练集
print('start read transSet')
trainDataArr, trainLabelArr = loadData('/home/litao/ML/slm/naive_Bayes/Mnist/mnist_train.csv')

start read transSet


In [10]:
# 获取测试集

print('start read testSet')
testDataArr, testLabelArr = loadData('/home/litao/ML/slm/naive_Bayes/Mnist/mnist_test.csv')

start read testSet


In [11]:
print('start to train')
Py, Px_y = getAllProbability(trainDataArr, trainLabelArr)

#使用习得的先验概率分布和条件概率分布对测试集进行测试
print('start to test')
accuracy = model_test(Py, Px_y, testDataArr, testLabelArr)

#打印准确率
print('the accuracy is:', accuracy)
#打印时间
print('time span:', time.time() -start)

start to train
start to test
the accuracy is: 0.8432999999999999
time span: 77.70210337638855


## 鸢尾花分类（高斯朴素贝叶斯）

In [216]:
import numpy as np
import pandas as pd
import random

In [217]:
dataSet =pd.read_csv('../titantic_and_iris_dataset-master/iris.csv')#,header = None
dataSet = dataSet.iloc[:, 1:]
dataSet.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [218]:
dataSet.index
#print(type(dataSet.index))
l = list(dataSet.index)
#l
dataSet.iloc[:,-1].value_counts().index

Index(['setosa', 'virginica', 'versicolor'], dtype='object')

In [187]:
dataSet.iloc[:,-1].value_counts()

setosa        50
virginica     50
versicolor    50
Name: Species, dtype: int64

In [188]:
idx = list(dataSet.index)  # 获得数据集的索引
random.shuffle(idx)  # 打乱索引
dataSet.index = idx  # 将dataSet数据集的索引改为idx

In [189]:
dataSet.head()  # 可以看出dataSet的索引为乱序

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
104,5.1,3.5,1.4,0.2,setosa
120,4.9,3.0,1.4,0.2,setosa
145,4.7,3.2,1.3,0.2,setosa
95,4.6,3.1,1.5,0.2,setosa
68,5.0,3.6,1.4,0.2,setosa


In [190]:
n = dataSet.shape[0]  # 总数据量
m = int(n * 0.8)  # 训练集的数量，因为数量是整数需要使用int()
train = dataSet.loc[range(m), :]  # dataSet中[0,m)的索引为训练集
test = dataSet.loc[range(m,n), :] #  dataSet中[m,n)的索引为测试集
test.index = range(test.shape[0]) #  将测试集的索引改为[0,shape[0]);

In [191]:
# 获得类别
labels = train.iloc[:, -1].value_counts().index# 类别
#type(labels)
labels

Index(['versicolor', 'setosa', 'virginica'], dtype='object')

In [192]:
li = labels.tolist()
li

['versicolor', 'setosa', 'virginica']

In [193]:
mean = []  # 每个类别的均差
std = []  # 每个类别方差
result = []  # 测试集预测结果

In [194]:
for i in labels:
    item = train.loc[train.iloc[:, -1] == i, :]
    m = item.iloc[:, :-1].mean()
    s = np.sum((item.iloc[:, :-1]-m)**2)/(item.shape[0])
    mean.append(m)
    std.append(s)
means = pd.DataFrame(mean, index = labels)
stds = pd.DataFrame(std, index = labels)

In [195]:
means

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,5.929268,2.785366,4.236585,1.317073
setosa,5.040000,3.477500,1.482500,0.247500
virginica,6.574359,2.930769,5.538462,2.020513


In [196]:
stds

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,0.252314,0.088566,0.208174,0.040440
setosa,0.102400,0.144244,0.028944,0.011494
virginica,0.409086,0.094438,0.333136,0.068297


In [197]:
test.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,4.9,3.0,1.4,0.2,setosa
1,7.2,3.2,6.0,1.8,virginica
2,4.4,3.2,1.3,0.2,setosa
3,6.5,3.2,5.1,2.0,virginica
4,5.0,2.0,3.5,1.0,versicolor


In [198]:
iset = test.iloc[2, :-1].tolist()
iset

[4.4, 3.2, 1.3, 0.2]

In [199]:
#广播
iset - means

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,-1.529268,0.414634,-2.936585,-1.117073
setosa,-0.640000,-0.277500,-0.182500,-0.047500
virginica,-2.174359,0.269231,-4.238462,-1.820513


In [200]:
#means['setosa']
test.iloc[:,-1]

0         setosa
1      virginica
2         setosa
3      virginica
4     versicolor
5     versicolor
6     versicolor
7         setosa
8      virginica
9     versicolor
10        setosa
11        setosa
12     virginica
13    versicolor
14    versicolor
15     virginica
16     virginica
17     virginica
18    versicolor
19        setosa
20     virginica
21     virginica
22        setosa
23     virginica
24        setosa
25        setosa
26    versicolor
27     virginica
28        setosa
29    versicolor
Name: Species, dtype: object

In [201]:
li[0]

'versicolor'

In [202]:
a = list(test.iloc[:,-1] == li[1]).count(True)
a

10

In [203]:
test.shape[1]-1

4

In [204]:
py = []
#'int' object is not iterable
for i in range(len(li)):
    py.append(np.sum(test.iloc[:,-1] == li[i])/len(test))
    
py

[0.3, 0.3333333333333333, 0.36666666666666664]

In [205]:
for i in range(3):
    py = list(test.iloc[:,-1] == li[i]).count(True) / test.shape[0]
    print(py)

0.3
0.3333333333333333
0.36666666666666664


In [206]:
a = [1,2,3]
b = np.array([1,2,3])
a * b

array([1, 4, 9])

In [207]:
for j in range(test.shape[0]):
    iset = test.iloc[j, :-1].tolist()
    iprob = np.exp(-1*(iset-means)**2/(2*stds))/np.sqrt(2*np.pi*stds)  # 根据正态分布公式
    prob = 1
    for k in range(test.shape[1]-1):
        #py = list(test.iloc[:,-1] == li[k]).count(True) / test.shape[0]
        prob = prob * iprob.iloc[:,k]
        prob *= py
    cla = prob.index[np.argmax(prob.values)]
    result.append(cla)
test['predict'] = result

In [208]:
iprob

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,0.686846,0.193712,7.401485e-01,1.311671e+00
setosa,0.001747,0.003669,1.144440e-68,8.565201e-30
virginica,0.525548,0.076811,1.369831e-01,2.100340e-01


In [209]:
iprob * 1

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,0.686846,0.193712,7.401485e-01,1.311671e+00
setosa,0.001747,0.003669,1.144440e-68,8.565201e-30
virginica,0.525548,0.076811,1.369831e-01,2.100340e-01


In [210]:
prob * py

versicolor     8.560870e-04
setosa        4.163955e-105
virginica      7.697457e-06
dtype: float64

In [212]:
train, test = randSplit(dataSet, 0.8)
gnb_classify(train, test)

模型预测准确率为0.9333333333333333


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,predict
0,5.1,3.5,1.4,0.3,setosa,setosa
1,7.4,2.8,6.1,1.9,virginica,virginica
2,4.9,3.0,1.4,0.2,setosa,setosa
3,6.7,3.3,5.7,2.5,virginica,virginica
4,5.0,2.3,3.3,1.0,versicolor,versicolor
5,5.0,3.2,1.2,0.2,setosa,setosa
6,5.1,3.8,1.6,0.2,setosa,setosa
7,5.5,2.4,3.7,1.0,versicolor,versicolor
8,6.9,3.1,4.9,1.5,versicolor,virginica
9,5.4,3.9,1.7,0.4,setosa,setosa


## 将上述代码整理的下面源码

In [219]:
#np.random.permutation(int(rate*df.shape【0】))，然后带着索引去找数据这样不是更方便？
def randSplit(dataSet, rate):
    l = list(dataSet.index)#提取出索引
    random.shuffle(l)#随机打乱索引
    dataSet.index = l #将打乱后的索引重新赋值给原数据集
    n = dataSet.shape[0]#总行数
    m = int(n * rate)#训练集的数量
    train = dataSet.loc[range(m), :]#提取前m个记录作为训练集
    test = dataSet.loc[range(m, n), :] #剩下的作为测试集
    dataSet.index = range(dataSet.shape[0]) #更新原数据集的索引
    test.index = range(test.shape[0]) #更新测试集的索引
    return train, test   

In [221]:
test.iloc[:,-1]

0         setosa
1         setosa
2      virginica
3         setosa
4      virginica
5      virginica
6         setosa
7         setosa
8     versicolor
9     versicolor
10     virginica
11        setosa
12     virginica
13     virginica
14        setosa
15        setosa
16    versicolor
17    versicolor
18        setosa
19        setosa
20        setosa
21    versicolor
22     virginica
23     virginica
24    versicolor
25     virginica
26    versicolor
27        setosa
28     virginica
29     virginica
Name: predict, dtype: object

In [222]:
def gnb_classify(train,test):
    labels = train.iloc[:,-1].value_counts().index #提取训练集的标签种类
    mean =[] #存放每个类别的均值
    std =[] #存放每个类别的方差
    result = [] #存放测试集的预测结果
    
    #缺少Py即先验概率，补上了
    py = []
    #'int' object is not iterable
    for i in range(len(li)):
        py.append(np.sum(test.iloc[:,-1] == li[i])/len(test))
        
        
        
    for i in labels:
        item = train.loc[train.iloc[:,-1]==i,:] #分别提取出每一种类别
        m = item.iloc[:,:-1].mean() #当前类别的平均值
        s = np.sum((item.iloc[:,:-1]-m)**2)/(item.shape[0]) #当前类别的方差
        mean.append(m) #将当前类别的平均值追加至列表
        std.append(s) #将当前类别的方差追加至列表
        
    means = pd.DataFrame(mean,index=labels) #变成DF格式，索引为类标签
    stds = pd.DataFrame(std,index=labels) #变成DF格式，索引为类标签
    
    for j in range(test.shape[0]):
        iset = test.iloc[j,:-1].tolist() #当前测试实例
        # means已经转换成DataFrame格式了，这里用到的是广播功能
        iprob = np.exp(-1*(iset-means)**2/(stds*2))/(np.sqrt(2*np.pi*stds)) #正态分布公式
        prob = 1 #初始化当前实例总概率
        for k in range(test.shape[1]-1): #遍历每个特征
            #缺少Py即先验概率
           
            #条件概率
            prob = prob * iprob.iloc[:,k] #特征概率  之积即为当前实例概率
            prob *= py
            cla = prob.index[np.argmax(prob.values)] #返回最大概率的类别
        result.append(cla)
    #新增一列
    test['predict']=result
    acc = (test.iloc[:,-1]==test.iloc[:,-2]).mean() #计算预测准确率
    print(f'模型预测准确率为{acc}')
    return test

In [223]:
for i in range(20):#测试20次，对比测试成功的概率
    train, test = randSplit(dataSet, 0.8)
    gnb_classify(train, test)

模型预测准确率为1.0
模型预测准确率为0.9333333333333333
模型预测准确率为1.0
模型预测准确率为0.9666666666666667
模型预测准确率为0.9666666666666667
模型预测准确率为0.9333333333333333
模型预测准确率为1.0
模型预测准确率为0.9
模型预测准确率为1.0
模型预测准确率为0.9333333333333333
模型预测准确率为0.9
模型预测准确率为0.9666666666666667
模型预测准确率为0.9666666666666667
模型预测准确率为0.9333333333333333
模型预测准确率为0.9333333333333333
模型预测准确率为0.9333333333333333
模型预测准确率为0.9333333333333333
模型预测准确率为0.9666666666666667
模型预测准确率为1.0
模型预测准确率为0.9666666666666667


## 社区评论是否为侮辱类词语（伯努利朴素贝叶斯）

In [225]:
from numpy import *
 
#加载数据
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]#类别标签向量，1代表侮辱性词汇，0代表非侮辱性词汇
    return postingList, classVec

In [226]:
##合并所有单词，利用set来去重，得到所有单词的唯一列表
def createVocabList(dataSet):
    vocabSet = set() #创建一个空的集合
    for doc in dataSet: #遍历dataSet中的每一条言论
        vocabSet = vocabSet | set(doc) #取并集
        vocabList = list(vocabSet)
    return vocabList

In [227]:
#获得训练集向量
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList) #创建一个其中所含元素都为0的向量
    for word in inputSet: #遍历每个词条
        if word in vocabList: #如果词条存在于词汇表中，则变为1
            returnVec[vocabList.index(word)] = 1
        else:
            print(f" {word} is not in my Vocabulary!" )
    return returnVec #返回文档向量

In [228]:
#生成训练集向量列表
def get_trainMat(dataSet):
    trainMat = [] #初始化向量列表
    vocabList = createVocabList(dataSet) #生成词汇表
    for inputSet in dataSet: #遍历样本词条中的每一条样本
        returnVec=setOfWords2Vec(vocabList, inputSet) #将当前词条向量化
        trainMat.append(returnVec) #追加到向量列表中
    return trainMat

In [231]:
#分类器训练函数
def trainNB(trainMat,classVec):
    n = len(trainMat) #计算训练的文档数目
    m = len(trainMat[0]) #计算每篇文档的词条数
    pAb = sum(classVec)/n #文档属于侮辱类的概率
    
    #下面的初始化是为了防止出现0概率
    p0Num = np.ones(m) #词条出现数初始化为1
    p1Num = np.ones(m) #词条出现数初始化为1
    p0Denom = 2 #分母初始化为2
    p1Denom = 2 #分母初始化为2
    for i in range(n): #遍历每一个文档
        if classVec[i] == 1: #统计属于侮辱类的条件概率所需的数据
            p1Num += trainMat[i]
            p1Denom += sum(trainMat[i])
        else: #统计属于非侮辱类的条件概率所需的数据
            p0Num += trainMat[i]
            p0Denom += sum(trainMat[i])
    p1V = np.log(p1Num/p1Denom)   # numpy数组 / float = 1中每个单词/1中总单词
    p0V = np.log(p0Num/p0Denom)    # 这里为什么还用ln来处理，后面说明
    return p0V,p1V,pAb #返回属于非侮辱类,侮辱类和文档属于侮辱类的概率

In [232]:
#朴素贝叶斯分类器函数
def classifyNB(vec2Classify, p0V, p1V, pAb):
    p1 = sum(vec2Classify * p1V) + np.log(pAb)    #对应元素相乘# 相乘后得到哪些单词存在，再求和，再+log(P(C))
    p0 = sum(vec2Classify * p0V) + np.log(1- pAb) #对应元素相乘# 由于使用的是ln，这里其实都是对数相加
    if p1 > p0:
        return 1
    else:
        return 0

In [233]:
#朴素贝叶斯测试函数
def testingNB(testVec):
    dataSet,classVec = loadDataSet() #创建实验样本
    vocabList = createVocabList(dataSet) #创建词汇表
    trainMat= get_trainMat(dataSet) #将实验样本向量化
    p0V,p1V,pAb = trainNB(trainMat,classVec) #训练朴素贝叶斯分类器
    thisone = setOfWords2Vec(vocabList, testVec) #测试样本向量化
    if classifyNB(thisone,p0V,p1V,pAb)==1:
        print(testVec,'属于侮辱类') #执行分类并打印分类结果
    else:
        print(testVec,'属于非侮辱类') #执行分类并打印分类结果

In [234]:
testVec1 = ['love', 'my', 'dalmation']
testingNB(testVec1)

['love', 'my', 'dalmation'] 属于非侮辱类


In [235]:
testVec2 = ['stupid', 'garbage']
testingNB(testVec2)

['stupid', 'garbage'] 属于侮辱类


In [224]:
[0] * 3

[0, 0, 0]

In [ ]:
"""#优化词集模型= 为 词袋模型+=，将单词列表变为数字向量列表
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)    #获得所有单词等长的0列表
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1   #对应单词位置加1
    return returnVec
 
# 返回的是0、1各自两个分类中每个单词数量除以该分类单词总量再取对数ln 以及0、1两类的比例
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  # 样本数
    numWords = len(trainMatrix[0])  # 特征数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 1类所占比例
    p0Num = ones(numWords)
    p1Num = ones(numWords)  #初始化所有单词为1
    p0Denom = 2.0
    p1Denom = 2.0  #初始化总单词为2        后面解释为什么这四个不初始化为0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:       #求1类
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]     #求0类
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)  # numpy数组 / float = 1中每个单词/1中总单词
    p0Vect = log(p0Num / p0Denom)  # 这里为什么还用ln来处理，后面说明
    return p0Vect, p1Vect, pAbusive
 
#P(X|C)判断各类别的概率大小（这里是0、1）
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)  # 相乘后得到哪些单词存在，再求和，再+log(P(C))
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1) # 由于使用的是ln，这里其实都是对数相加
    if p1 > p0:
        return 1
    else:
        return 0"""
 

In [236]:
"""##封装调用的函数
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(bagOfWords2VecMN(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))
    #上面求出了0、1两个类中各单词所占该类的比例，以及0、1的比例
 
    #下面是预测两条样本数据的类别
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(bagOfWords2VecMN(myVocabList, testEntry)) #先将测试数据转为numpy的词袋模型 [0 2 0 5 1 0 0 3 ...]
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)) #传值判断
 
    testEntry = ['stupid', 'garbage']
    thisDoc = array(bagOfWords2VecMN(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
 """



"##封装调用的函数\ndef testingNB():\n    listOPosts, listClasses = loadDataSet()\n    myVocabList = createVocabList(listOPosts)\n    trainMat = []\n    for postinDoc in listOPosts:\n        trainMat.append(bagOfWords2VecMN(myVocabList, postinDoc))\n    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))\n    #上面求出了0、1两个类中各单词所占该类的比例，以及0、1的比例\n \n    #下面是预测两条样本数据的类别\n    testEntry = ['love', 'my', 'dalmation']\n    thisDoc = array(bagOfWords2VecMN(myVocabList, testEntry)) #先将测试数据转为numpy的词袋模型 [0 2 0 5 1 0 0 3 ...]\n    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)) #传值判断\n \n    testEntry = ['stupid', 'garbage']\n    thisDoc = array(bagOfWords2VecMN(myVocabList, testEntry))\n    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))\n "